In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,NaN,NaN,...,NaN,8.284196,3.647166,4018.882255,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,2158.499000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.053857,913.204865,...,2.592038,8.209520,3.582935,4010.031842,2.055103e+08,1.072430e+07,17.625670,1.858242e+08,0.778118,2259.910000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,18.820083,913.050400,...,2.457740,8.186736,3.430058,4028.768630,2.080841e+08,1.084112e+07,17.657353,1.863343e+08,0.779266,2225.422000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,20.603352,912.929616,...,2.412149,8.093445,3.364859,4330.599545,2.106490e+08,1.096989e+07,17.713982,1.870506e+08,0.780317,2203.646000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,21.109323,912.790145,...,2.331749,8.065900,3.446867,4749.071195,2.130490e+08,1.112092e+07,17.772465,1.877383e+08,0.781274,2382.720000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,17.495624,912.621888,...,2.234563,8.033948,3.612886,5058.377260,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.782138,3001.521000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,16.616854,912.464594,...,2.154053,8.063370,3.874729,5346.288081,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.782910,3092.068000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,17.909653,912.481188,...,2.092206,8.026843,4.308613,5527.794738,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.783594,3749.448000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,18.651220,912.629014,...,2.043673,8.013806,4.707700,5623.811148,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.784190,4174.942000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,19.304559,912.778525,...,1.941253,6.472496,5.027391,5779.044240,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.784389,4522.969000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,NaN,NaN,NaN,...,NaN,NaN,8.284196,3.647166,4018.882255,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.053857,913.204865,1076.210957,...,69.530399,2.592038,8.209520,3.582935,4010.031842,2.055103e+08,1.072430e+07,17.625670,1.858242e+08,0.778118
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,18.820083,913.050400,1112.994688,...,73.576779,2.457740,8.186736,3.430058,4028.768630,2.080841e+08,1.084112e+07,17.657353,1.863343e+08,0.779266
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,20.603352,912.929616,1158.198474,...,75.010305,2.412149,8.093445,3.364859,4330.599545,2.106490e+08,1.096989e+07,17.713982,1.870506e+08,0.780317
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,21.109323,912.790145,1192.488712,...,76.367662,2.331749,8.065900,3.446867,4749.071195,2.130490e+08,1.112092e+07,17.772465,1.877383e+08,0.781274
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,17.495624,912.621888,1227.235440,...,78.100902,2.234563,8.033948,3.612886,5058.377260,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.782138
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,16.616854,912.464594,1218.184435,...,78.279905,2.154053,8.063370,3.874729,5346.288081,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.782910
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,17.909653,912.481188,1194.341210,...,76.333321,2.092206,8.026843,4.308613,5527.794738,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.783594
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,18.651220,912.629014,1182.566664,...,74.627583,2.043673,8.013806,4.707700,5623.811148,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.784190
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,19.304559,912.778525,1176.573047,...,73.871522,1.941253,6.472496,5.027391,5779.044240,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.784389


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2259.910000
1     2225.422000
2     2203.646000
3     2382.720000
4     3001.521000
5     3092.068000
6     3749.448000
7     4174.942000
8     4522.969000
9     4578.716000
10    4341.821857
11    4108.786286
12    3869.334857
13    3764.518000
14    3773.354000
15    3786.974000
16    4235.532000
17    4784.413000
18    4538.256000
19            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [24]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.109126,-1.002190,-1.667898,-1.603182,-2.155349,-1.836009,-1.727113,-2.028667
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,0.577311,1.216186,-2.561501,...,-2.422536,2.084009,0.950765,-1.087231,-1.679381,-1.407610,-1.617234,-1.606661,-1.487031,-1.640040
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.716511,0.584049,-1.543286,...,-0.461057,1.533478,0.902449,-1.289637,-1.655070,-1.227554,-1.085281,-1.405566,-1.297277,-1.282141
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,0.316374,0.089749,-0.291994,...,0.233843,1.346588,0.704613,-1.375961,-1.263451,-1.048118,-0.498915,-1.046128,-1.030811,-0.954387
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,0.609437,-0.481022,0.657199,...,0.891822,1.017001,0.646198,-1.267383,-0.720493,-0.880219,0.188846,-0.674928,-0.774984,-0.656193
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-1.483650,-1.169601,1.619027,...,1.732008,0.618603,0.578439,-1.047577,-0.319175,-0.753036,0.678037,-0.621083,-0.720881,-0.386975
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-1.992642,-1.813314,1.368485,...,1.818780,0.288568,0.640834,-0.700901,0.054383,-0.650968,1.147222,-0.576797,-0.697501,-0.146150
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-1.243841,-1.745403,0.708478,...,0.875174,0.035037,0.563373,-0.126446,0.289884,-0.263323,1.385758,0.045699,-0.160026,0.066866
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.814318,-1.140440,0.382545,...,0.048319,-0.163914,0.535726,0.401938,0.414463,0.099685,1.394217,0.561610,0.303092,0.252658
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-0.435899,-0.528578,0.216635,...,-0.318181,-0.583768,-2.732837,0.825204,0.615874,0.425785,1.182549,0.843135,0.664950,0.314660


In [25]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2259.910000
1     2225.422000
2     2203.646000
3     2382.720000
4     3001.521000
5     3092.068000
6     3749.448000
7     4174.942000
8     4522.969000
9     4578.716000
10    4341.821857
11    4108.786286
12    3869.334857
13    3764.518000
14    3773.354000
15    3786.974000
16    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [26]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.019551,0.255662,-0.068592,...,-2.040988,-1.105857,-1.059306,1.648055,1.965957,0.815772,-0.261923,0.400377,0.581416,-0.093694


In [27]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [28]:
train_input.iloc[-(len(train_input)//5):]

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
14,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.112415,1.055738,-0.215514,...,-0.475597,-0.794493,-0.441264,0.516338,0.775682,1.201787,-0.478243,0.912412,1.048657,1.309955
15,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.876169,1.185514,0.366817,...,-0.567957,-1.040705,-0.680946,0.682550,1.013605,1.168595,-0.457848,0.842867,0.969305,1.043055
16,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,0.823650,0.919076,0.654201,...,-0.967708,-1.063870,-0.832798,1.136575,1.594449,1.062470,-0.378078,0.681669,0.840429,0.525187


In [29]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [30]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [31]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3352322572, 4109310573, 3947766957, 1536034505, 2393913794, 3205970455, 1132486815, 1603047388, 3234057586, 1132691934, 219929125, 736473399, 4294126692, 374800985, 2675734343, 2599791810, 1959640765, 3868084114, 930426292, 3225869626, 3346651922, 3011255741, 962875017, 3686347245, 3666982130, 1300321600, 1308001756, 2803444626, 2271046317, 2426950031, 2777934982, 4172197075, 1973360205, 3792208199, 1166475276, 2501799683, 1966667705, 2990304181, 228918162, 4002518774, 1284968454, 4043227710, 2898088106, 629049175, 2817141863, 1183656666, 2126886510, 579629568, 1952953086, 3688959595, 2872980164, 259811782, 511780445, 2054954536, 1355412988, 2568720704, 771870714, 2959030556, 1646394210, 2618933714, 4070928320, 698698885, 918998953, 2106037490, 1253846615, 3126253741, 3645486059, 2857145491, 2114139663, 775804368, 2969175745, 148814866, 627444870, 3978370862, 3002258493, 1662855018, 3246642273, 2171427182, 420680730, 1395246361, 496574184, 4056959164, 832185054, 4046994773, 3264194672

val_loss: 46110.96875


Step: 88 ___________________________________________
val_loss: 46110.95703125


Step: 89 ___________________________________________
val_loss: 46110.95703125


Step: 90 ___________________________________________
val_loss: 18367.814453125


Step: 91 ___________________________________________
val_loss: 46111.703125


Step: 92 ___________________________________________
val_loss: 46111.02734375


Step: 93 ___________________________________________
val_loss: 19086.294921875


Step: 94 ___________________________________________
val_loss: 3972.419677734375


Step: 95 ___________________________________________
val_loss: 46148.77734375


Step: 96 ___________________________________________
val_loss: 46111.01171875


Step: 97 ___________________________________________
val_loss: 31924.34375


Step: 98 ___________________________________________
val_loss: 15921.5888671875


Step: 99 ___________________________________________
val_loss: 46111.65234375


final_seed: 39

In [32]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 476ms/step - loss: 13145155.0000 - val_loss: 15123875.0000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 12832391.0000 - val_loss: 10080284.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 11458501.0000 - val_loss: 3754350.2500
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 9407000.0000 - val_loss: 5169909.5000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 9044586.0000 - val_loss: 4059414.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 8941931.0000 - val_loss: 4645490.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 7383219.0000 - val_loss: 5020238.5000
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 6659216.5000 - val_loss: 4489838.5000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 8986936.0000 - val_loss:

1/1 [==============================] - 0s 14ms/step - loss: 969708.1250 - val_loss: 596756.4375
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 794813.3125 - val_loss: 1044195.8125
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 817299.1250 - val_loss: 428063.7500
Epoch 148/10000
1/1 [==============================] - 0s 15ms/step - loss: 679528.7500 - val_loss: 610301.5625
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 741323.7500 - val_loss: 789188.6875
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 825439.1250 - val_loss: 606369.3125
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 651423.6875 - val_loss: 755522.9375
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 759715.5000 - val_loss: 994291.0000
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 936387.0000 - val_loss: 1007822.5000
Epoch 

1/1 [==============================] - 0s 14ms/step - loss: 194887.3750 - val_loss: 30762.4902
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 188320.9062 - val_loss: 37920.5742
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 165283.0469 - val_loss: 12371.3867
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 326558.8438 - val_loss: 128498.2188
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 142247.7031 - val_loss: 158410.6094
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 212162.5469 - val_loss: 91555.1172
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 328170.7188 - val_loss: 126651.8047
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 209327.4219 - val_loss: 133243.9219
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 150886.4531 - val_loss: 154816.1250
Epoch 300/10

Epoch 438/10000
1/1 [==============================] - 0s 14ms/step - loss: 142874.9531 - val_loss: 42565.0273
Epoch 439/10000
1/1 [==============================] - 0s 14ms/step - loss: 163899.9688 - val_loss: 78688.9297
Epoch 440/10000
1/1 [==============================] - 0s 14ms/step - loss: 128588.5312 - val_loss: 108098.5703
Epoch 441/10000
1/1 [==============================] - 0s 14ms/step - loss: 167489.1719 - val_loss: 89026.9141
Epoch 442/10000
1/1 [==============================] - 0s 14ms/step - loss: 116638.1172 - val_loss: 51791.1523
Epoch 443/10000
1/1 [==============================] - 0s 14ms/step - loss: 114548.2578 - val_loss: 60956.9570
Epoch 444/10000
1/1 [==============================] - 0s 14ms/step - loss: 70443.9766 - val_loss: 37159.6406
Epoch 445/10000
1/1 [==============================] - 0s 14ms/step - loss: 90824.7266 - val_loss: 58013.8438
Epoch 446/10000
1/1 [==============================] - 0s 15ms/step - loss: 192397.7031 - val_loss: 50313.3945
Ep

1/1 [==============================] - 0s 14ms/step - loss: 135038.7344 - val_loss: 25613.8906
Epoch 587/10000
1/1 [==============================] - 0s 14ms/step - loss: 83745.4844 - val_loss: 22533.8691
Epoch 588/10000
1/1 [==============================] - 0s 14ms/step - loss: 175761.0625 - val_loss: 20885.8105
Epoch 589/10000
1/1 [==============================] - 0s 14ms/step - loss: 130310.6797 - val_loss: 36236.1172
Epoch 590/10000
1/1 [==============================] - 0s 14ms/step - loss: 145911.3281 - val_loss: 18511.7617
Epoch 591/10000
1/1 [==============================] - 0s 14ms/step - loss: 149606.4062 - val_loss: 20883.8320
Epoch 592/10000
1/1 [==============================] - 0s 14ms/step - loss: 217630.2969 - val_loss: 17585.0820
Epoch 593/10000
1/1 [==============================] - 0s 14ms/step - loss: 138783.2500 - val_loss: 20717.7715
Epoch 594/10000
1/1 [==============================] - 0s 14ms/step - loss: 98386.1172 - val_loss: 46384.2656
Epoch 595/10000
1/1

Epoch 735/10000
1/1 [==============================] - 0s 15ms/step - loss: 118327.1016 - val_loss: 1070.4327
Epoch 736/10000
1/1 [==============================] - 0s 15ms/step - loss: 109631.3672 - val_loss: 4927.5405
Epoch 737/10000
1/1 [==============================] - 0s 14ms/step - loss: 222162.0156 - val_loss: 3845.4714
Epoch 738/10000
1/1 [==============================] - 0s 14ms/step - loss: 206292.4219 - val_loss: 2188.4258
Epoch 739/10000
1/1 [==============================] - 0s 14ms/step - loss: 167828.4062 - val_loss: 3446.0066
Epoch 740/10000
1/1 [==============================] - 0s 14ms/step - loss: 180379.6250 - val_loss: 9520.2002
Epoch 741/10000
1/1 [==============================] - 0s 14ms/step - loss: 113800.9766 - val_loss: 1009.6406
Epoch 742/10000
1/1 [==============================] - 0s 14ms/step - loss: 195935.0312 - val_loss: 2257.2048
Epoch 743/10000
1/1 [==============================] - 0s 14ms/step - loss: 155666.5625 - val_loss: 567.0596
Epoch 744/1

1/1 [==============================] - 0s 14ms/step - loss: 241505.7969 - val_loss: 4711.8413
Epoch 884/10000
1/1 [==============================] - 0s 15ms/step - loss: 103691.5547 - val_loss: 17305.7539
Epoch 885/10000
1/1 [==============================] - 0s 14ms/step - loss: 154857.5156 - val_loss: 5706.7598
Epoch 886/10000
1/1 [==============================] - 0s 15ms/step - loss: 107052.0859 - val_loss: 10388.5771
Epoch 887/10000
1/1 [==============================] - 0s 14ms/step - loss: 243483.8750 - val_loss: 25619.3730
Epoch 888/10000
1/1 [==============================] - 0s 14ms/step - loss: 237527.3750 - val_loss: 6685.5942
Epoch 889/10000
1/1 [==============================] - 0s 14ms/step - loss: 163971.6250 - val_loss: 17559.3027
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 144450.7500 - val_loss: 12140.6494
Epoch 891/10000
1/1 [==============================] - 0s 15ms/step - loss: 210303.5156 - val_loss: 10916.2588
Epoch 892/10000
1/1 

1/1 [==============================] - 0s 14ms/step - loss: 87646.7422 - val_loss: 7876.9316
Epoch 1032/10000
1/1 [==============================] - 0s 14ms/step - loss: 129404.8516 - val_loss: 9540.8545
Epoch 1033/10000
1/1 [==============================] - 0s 14ms/step - loss: 141554.6875 - val_loss: 17066.7539
Epoch 1034/10000
1/1 [==============================] - 0s 14ms/step - loss: 178947.3906 - val_loss: 5713.8140
Epoch 1035/10000
1/1 [==============================] - 0s 14ms/step - loss: 87065.9375 - val_loss: 11087.2510
Epoch 1036/10000
1/1 [==============================] - 0s 14ms/step - loss: 147964.8281 - val_loss: 6023.6665
Epoch 1037/10000
1/1 [==============================] - 0s 14ms/step - loss: 169175.3438 - val_loss: 6581.2554
Epoch 1038/10000
1/1 [==============================] - 0s 14ms/step - loss: 170025.7812 - val_loss: 6287.1099
Epoch 1039/10000
1/1 [==============================] - 0s 14ms/step - loss: 162393.3125 - val_loss: 5905.4062
Epoch 1040/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 176247.6250 - val_loss: 52052.0156
Epoch 1178/10000
1/1 [==============================] - 0s 15ms/step - loss: 229263.2500 - val_loss: 52237.3555
Epoch 1179/10000
1/1 [==============================] - 0s 14ms/step - loss: 241065.8125 - val_loss: 46111.7148
Epoch 1180/10000
1/1 [==============================] - 0s 14ms/step - loss: 255622.7969 - val_loss: 48225.6875
Epoch 1181/10000
1/1 [==============================] - 0s 14ms/step - loss: 254140.3281 - val_loss: 47155.3164
Epoch 1182/10000
1/1 [==============================] - 0s 14ms/step - loss: 140727.4062 - val_loss: 49674.9180
Epoch 1183/10000
1/1 [==============================] - 0s 15ms/step - loss: 277823.0938 - val_loss: 52029.0430
Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 113127.6016 - val_loss: 48210.0156
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 126133.7344 - val_loss: 46440.6875
Epoch 118

In [33]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[4784.413](test_target) - [[3777.0833]](prediction) = 1007.3297480468746


In [34]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [35]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.854322,-0.039492,-1.053361,-1.535732,-1.563966,-1.486447,-1.569376,-1.670526
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.153604,1.562933,-1.778134,...,-2.058283,1.629182,1.008433,-0.263029,-1.068524,-1.054354,-1.123510,-1.094963,-1.060326,-1.106095
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.156528,0.978457,-1.069590,...,-0.618292,0.810058,0.750349,-0.795068,-1.036424,-0.611167,-0.688099,-0.751705,-0.657986,-0.586293
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,0.889378,0.521425,-0.198856,...,-0.108141,0.531989,-0.306402,-1.021975,-0.519322,-0.169507,-0.208149,-0.138167,-0.092993,-0.110272
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,1.186136,-0.006314,0.461657,...,0.374904,0.041602,-0.618426,-0.736570,0.197610,0.243755,0.354794,0.495450,0.449443,0.322816
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.933338,-0.642977,1.130963,...,0.991714,-0.551167,-0.980361,-0.158794,0.727518,0.556802,0.755205,0.587361,0.564158,0.713819
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-1.448746,-1.238157,0.956619,...,1.055416,-1.042220,-0.647075,0.752467,1.220771,0.808031,1.139240,0.662954,0.613731,1.063586
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-0.690505,-1.175366,0.497341,...,0.362682,-1.419444,-1.060840,2.262462,1.531732,1.762172,1.334485,1.725517,1.753350,1.372965


train_target:


,Paraná - Consumo de Cimento (t)
0,2259.910
1,2225.422
2,2203.646
3,2382.720
4,3001.521
5,3092.068
6,3749.448
7,4174.942


test_input:


...

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.019551,0.255662,-0.068592,...,-2.040988,-1.105857,-1.059306,1.648055,1.965957,0.815772,-0.261923,0.400377,0.581416,-0.093694


test_target:


,Paraná - Consumo de Cimento (t)
17,4784.413


1/1 [==============================] - 0s 37ms/step
Error: 1007.3297480468746




[3695.50439453125,
 4056.084228515625,
 4328.681640625,
 4393.142578125,
 4212.87451171875,
 3992.580810546875,
 3890.77734375,
 3899.6328125,
 3772.369140625,
 3777.083251953125]

In [36]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[3695.50439453125,
 4056.084228515625,
 4328.681640625,
 4393.142578125,
 4212.87451171875,
 3992.580810546875,
 3890.77734375,
 3899.6328125,
 3772.369140625,
 3777.083251953125]

In [37]:
mae = mean_absolute_error(predictions, targets)
mae

391.9770114676339

In [38]:
porcentage = mae/np.mean(targets)
porcentage

0.0938498010728748